In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import numpy as np

pd.set_option('display.max_columns', None)

In [4]:
import pandas as pd
import sys
import os

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.abspath('..'))

# Import the utility function for one-time directory change
from src.utilities.path_utilities import ensure_parent_dir

# Change to parent directory (only happens once even if cell is re-executed)
ensure_parent_dir('notebook_setup')

Original directory: /Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon/notebooks
Changed directory to: /Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon


'/Users/adrianhajdukiewicz/projects/private/2025_data_biz_hackathon'

In [5]:
pois = gpd.read_file("data/mazowieckie_all/gis_osm_pois_free_1.shp", encoding="utf-8")

In [6]:
pois_a = gpd.read_file("data/mazowieckie_all/gis_osm_pois_a_free_1.shp", encoding="utf-8")

In [7]:
pois_a = pois_a.to_crs(epsg=2180)

In [8]:
pois_a["centroid"] = pois_a.geometry.centroid

In [9]:
pois_a["geometry"] = pois_a["centroid"].to_crs(epsg=4326)

In [10]:
pois_a = pois_a.drop(columns=["centroid"])

In [11]:
pois

,osm_id,code,fclass,name,geometry
0,30776210,2301,restaurant,Dziupla,POINT (20.76933 52.3084)
1,31005854,2301,restaurant,None,POINT (20.94595 52.17691)
2,31156693,2502,bakery,Galeria Wypieków Lubaszka,POINT (20.95489 52.271)
3,31917380,2724,memorial,Pomnik Tysiąclecia Jazdy Polskiej,POINT (21.01451 52.21653)
4,38202108,2724,memorial,Pomnik polskich kryptologów,POINT (21.03313 52.12883)
...,...,...,...,...,...
98215,12847077865,2902,bench,None,POINT (21.11404 51.3935)
98216,12847077866,2902,bench,None,POINT (21.11415 51.39326)
98217,12847516473,2033,recycling_clothes,None,POINT (21.02689 52.18346)
98218,12847549932,2421,shelter,None,POINT (20.45471 52.28655)


In [12]:
pois_a

,osm_id,code,fclass,name,geometry
0,4943389,2204,park,Ogród Saski,POINT (21.00815 52.24062)
1,4999161,2204,park,Skwer Antoniego Słonimskiego,POINT (21.01176 52.20551)
2,5000672,2204,park,Park Szczęśliwicki,POINT (20.95942 52.20543)
3,5003248,2204,park,"Skwer Stanisława Broniewskiego ""Orszy""",POINT (21.02247 52.20313)
4,5032842,2204,park,Park Dreszera,POINT (21.02152 52.19746)
...,...,...,...,...,...
37891,1387801358,2252,pitch,None,POINT (21.25041 51.4655)
37892,1387801359,2252,pitch,None,POINT (21.25087 51.46521)
37893,1387801361,2082,school,Publiczna Szkoła Podstawowa im. Tadeusza Kości...,POINT (21.25029 51.46558)
37894,1387818672,2421,shelter,None,POINT (20.97839 51.39068)


In [13]:
result = gpd.GeoDataFrame(pd.concat([pois, pois_a], ignore_index=True), geometry='geometry')

In [14]:
result

,osm_id,code,fclass,name,geometry
0,30776210,2301,restaurant,Dziupla,POINT (20.76933 52.3084)
1,31005854,2301,restaurant,None,POINT (20.94595 52.17691)
2,31156693,2502,bakery,Galeria Wypieków Lubaszka,POINT (20.95489 52.271)
3,31917380,2724,memorial,Pomnik Tysiąclecia Jazdy Polskiej,POINT (21.01451 52.21653)
4,38202108,2724,memorial,Pomnik polskich kryptologów,POINT (21.03313 52.12883)
...,...,...,...,...,...
136111,1387801358,2252,pitch,None,POINT (21.25041 51.4655)
136112,1387801359,2252,pitch,None,POINT (21.25087 51.46521)
136113,1387801361,2082,school,Publiczna Szkoła Podstawowa im. Tadeusza Kości...,POINT (21.25029 51.46558)
136114,1387818672,2421,shelter,None,POINT (20.97839 51.39068)


In [15]:
result["lon"] = result.geometry.x
result["lat"] = result.geometry.y
result_clean = result[["fclass", "lon", "lat"]]
result_clean["x"] = np.floor((result_clean["lon"] - 54) / 0.01)
result_clean["y"] = np.floor((result_clean["lat"] - 30) / 0.02)
result_clean["key"] = result_clean["x"].astype(str) + "_" + result_clean["y"].astype(str)
result_clean["counter"] = 1
result_clean

/var/folders/76/rbsll_ws0ts8dy9ypd652xzr0000gp/T/ipykernel_41857/2434971968.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_clean["x"] = np.floor((result_clean["lon"] - 54) / 0.01)
/var/folders/76/rbsll_ws0ts8dy9ypd652xzr0000gp/T/ipykernel_41857/2434971968.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_clean["y"] = np.floor((result_clean["lat"] - 30) / 0.02)
/var/folders/76/rbsll_ws0ts8dy9ypd652xzr0000gp/T/ipykernel_41857/2434971968.py:6: SettingWithCopyWarning: 
A value is trying to 

,fclass,lon,lat,x,y,key,counter
0,restaurant,20.769334,52.308402,-3324.0,1115.0,-3324.0_1115.0,1
1,restaurant,20.945950,52.176908,-3306.0,1108.0,-3306.0_1108.0,1
2,bakery,20.954893,52.271002,-3305.0,1113.0,-3305.0_1113.0,1
3,memorial,21.014506,52.216532,-3299.0,1110.0,-3299.0_1110.0,1
4,memorial,21.033132,52.128829,-3297.0,1106.0,-3297.0_1106.0,1
...,...,...,...,...,...,...,...
136111,pitch,21.250405,51.465496,-3275.0,1073.0,-3275.0_1073.0,1
136112,pitch,21.250872,51.465212,-3275.0,1073.0,-3275.0_1073.0,1
136113,school,21.250288,51.465578,-3275.0,1073.0,-3275.0_1073.0,1
136114,shelter,20.978386,51.390675,-3303.0,1069.0,-3303.0_1069.0,1
